In [17]:
import pandas as pd

# Load the dataset from the CSV file
data = pd.read_csv("C:/Users/samue/Downloads/data_assessment_2/data_for_info_retriev_extract/Dataset_IMDB.csv")

# Define the three specific movies
movies = ['The Lion King', 'Star Wars', 'Starship Troopers']

# Initialize dictionaries to store the results
ir_results = {}
ie_results = {}

# Implement the information retrieval system
for movie in movies:
    ir_results[movie] = {'Mentioned': [], 'Reviewed': []}
    for index, row in data.iterrows():
        if movie.lower() in row['Text'].lower():
            if 'review' in row['Text'].lower():
                ir_results[movie]['Reviewed'].append(row['Id'])
            else:
                ir_results[movie]['Mentioned'].append(row['Id'])

# Display the results of the information retrieval system
for movie in movies:
    print(f"Mentions of {movie}: {ir_results[movie]['Mentioned']}")
    print(f"Reviews of {movie}: {ir_results[movie]['Reviewed']}")


Mentions of The Lion King: [12429, 2262, 3098, 4464, 8264, 2758, 3163, 9837, 3768, 2661, 9751, 2261, 2670, 15087, 23535, 10023, 13316, 22231, 9609, 18890]
Reviews of The Lion King: [25950, 2727, 2236, 5513, 9778, 12950, 7218, 5691, 24533, 12936, 18866, 23805, 7836, 9703, 3532, 4981, 3558, 3568, 3792, 5790, 8103, 8287, 3535, 5369]
Mentions of Star Wars: [4550, 7309, 12547, 2582, 12057, 3785, 3023, 2035, 4125, 11641, 4634, 7065, 7015, 7096, 7624, 11025, 5616, 6949, 7676, 3040, 12890, 15031, 21544, 24551, 24563, 12602, 9638, 10230, 11468, 13536, 13636, 21610, 18779, 22991, 25302, 25766, 10897, 20096]
Reviews of Star Wars: [26895, 22584, 24561, 18485, 20185, 2985, 5574, 3742, 6782, 16619, 24553, 2483, 25020, 5542, 18433, 18767, 7114, 7533, 6829, 6979, 7197, 27812, 28755, 28772, 6417, 6383, 18265, 3852, 6348, 7163, 8245, 28787, 5100, 6971, 7536, 29716, 5049, 5055, 5592, 7212]
Mentions of Starship Troopers: [10859, 12224, 11808, 9654, 11987, 11595, 12012, 14580, 11468, 13629, 9915, 13316, 25

In [18]:
# Define the directors of the movies
directors = {
    "The Lion King": "Roger Allers and Rob Minkoff",
    "Star Wars": "George Lucas",
    "Starship Troopers": "Paul Verhoeven"
}

# Calculate the average rating for each movie
average_ratings = {}

for movie in movies:
    movie_reviews = df[df["Text"].str.contains(movie)]
    average_rating = movie_reviews["Rating"].mean()
    average_ratings[movie] = average_rating

# Print the director and average rating for each movie
for movie in movies:
    director = directors[movie]
    rating = average_ratings[movie]
    print(f"{movie}:")
    print(f"Director: {director}")
    print(f"Average Rating: {rating}")
    print()



The Lion King:
Director: Roger Allers and Rob Minkoff
Average Rating: 0.65

Star Wars:
Director: George Lucas
Average Rating: 0.6666666666666666

Starship Troopers:
Director: Paul Verhoeven
Average Rating: 0.7



In [16]:

import pyttsx3

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Define a function to generate the voice message
def generate_voice_message(movie_title, director, num_reviews, num_mentions, avg_score):
    # Define the message template
    message = f"You have required information about {movie_title} movie, the director of this movie is {director}, there is a total of {num_reviews} reviews of this movie in the database, there are {num_mentions} mentions of this movie in other movie reviews. The average review score for this movie is {avg_score}, which makes it a {'BAD' if avg_score >= 6 else 'GOOD'} recommendation to watch."
    # Set the properties of the voice message
    engine.setProperty('rate', 150) # Set the speaking rate
    engine.setProperty('volume', 1) # Set the volume
    # Generate the voice message
    engine.say(message)
    engine.runAndWait()

# Call the function to generate the voice message
generate_voice_message("The Lion King", "Roger Allers and Rob Minkoff", 24, 20, 0.7)
